In [ ]:
import re
import requests
from bs4 import BeautifulSoup

# 与えられた日付の解析ページ(p)から、データを抽出して保存する関数
def etl(p):
	data = {}

	# 日付の抽出
	d_pattern = r"(\d+)月(\d+)日\(([土日月火水木金])+\)"
	d_src = p.select('.left-style')
	date = re.findall(d_pattern, d_src[0].text)[0]
	data["date"] = "%s-%s(%s)" % (date[0], date[1], date[2])
	print("=====" + data["date"] + "=====")

    # 天気予報の抽出
	weather           = p.select('.weather-telop')[0]
	high_temp         = p.select("[class='high-temp temp']")[0]
	high_temp_diff    = p.select("[class='high-temp tempdiff']")[0]
	low_temp          = p.select("[class='low-temp temp']")[0]
	low_temp_diff     = p.select("[class='low-temp tempdiff']")[0]
	rain_probability  = p.select('.rain-probability > td')
	wind_wave         = p.select('.wind-wave > td')[0]

    # データの保存
	data["forecasts"] = []
	forecast = {}
	forecast["weather"] = weather.text.strip()
	forecast["high_temp"] = high_temp.text.strip()
	forecast["high_temp_diff"] = high_temp_diff.text.strip()
	forecast["low_temp"] = low_temp.text.strip()
	forecast["low_temp_diff"] = low_temp_diff.text.strip()
	every_6h = {}
	for i in range(4):
		time_from = 0+6*i
		time_to   = 6+6*i
		itr       = '{:02}-{:02}'.format(time_from,time_to)
		every_6h[itr] = rain_probability[i].text.strip()
	forecast["rain_probability"] = every_6h
	forecast["wind_wave"] = wind_wave.text.strip()

	data["forecasts"].append(forecast)

	print(
		"天気              ： " + forecast["weather"] + "\n"
		"最高気温(C)       ： " + forecast["high_temp"] + "\n"
		"最高気温差(C)     ： " + forecast["high_temp_diff"] + "\n"
		"最低気温(C)       ： " + forecast["low_temp"] + "\n"
		"最低気温差(C)     ： " + forecast["low_temp_diff"] + "\n"
		"降水確率[00-06]   ： " + forecast["rain_probability"]['00-06'] + "\n"
		"降水確率[06-12]   ： " + forecast["rain_probability"]['06-12'] + "\n"
		"降水確率[12-18]   ： " + forecast["rain_probability"]['12-18'] + "\n"
		"降水確率[18-24]   ： " + forecast["rain_probability"]['18-24'] + "\n"
		"風向              ： " + forecast["wind_wave"] + "\n"
	)

	return data
# 関数はここまで

# 「pythonファイル名.py」形式で実行されているかどうかの判定
if __name__ == '__main__':

	# これ以降の処理を実行

	# アクセスするURL：日本気象協会 tenki.jp 大阪府八尾市を設定する
	url = 'https://tenki.jp/forecast/6/30/6200/27212/'

	# URLにアクセスしデータを取得する。結果は「resp1」に格納する
	resp1 = requests.get(url)

	#「resp1」のエンコーディング方式（UTF-8）で、
	#「resp1.text」をエンコードし「html」に格納する
	html =  resp1.text.encode(resp1.encoding)

	#BeautifulSoup のパーサー「lxml：高速処理」を利用して解析ページを作成する
	resp2 = BeautifulSoup(html, 'lxml')

	# 解析ページを保存するテーブルを用意する
	kaiseki = {}

	# 予測地点を取得する要素のパターンを指定する
	y_pattern = r"(.+)の今日明日の天気"

	# 指定した要素にある「title」から地域名を取得する
	kaiseki['tiiki'] = re.findall(y_pattern, resp2.title.text)[0]

	# 取得した地域名を表示する
	print(kaiseki['tiiki'] + "の天気")

	# 関数に渡すパラメータに今日の「天気予報ページ」をセット
	p_tdy = resp2.select('.today-weather')[0]
	# 関数に渡すパラメータに明日の「天気予報ページ」をセット
	p_tmr = resp2.select('.tomorrow-weather')[0]

	# 関数を使って今日のデータ抽出して保存
	kaiseki["today"] = etl(p_tdy)
	# 関数を使って明日のデータ抽出して保存
	kaiseki["tomorrow"] = etl(p_tmr)